In [1]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets.samples_generator import make_blobs
import matplotlib.pyplot as plt
from itertools import cycle
import pandas as pd    
import math

In [5]:
column_names = ['id','user_id','venue_id','latitude','longitude','created_at']
data = pd.read_csv('checkins.csv', sep=',', header=1 , names = column_names)
data.replace('                   ', np.nan, inplace=True)
data.dropna(inplace=True)
data.index = range(1,len(data) + 1)
data.head()

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
3,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
4,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
5,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18


In [6]:
exp_data = data[:100000]
exp_data = exp_data[['latitude', 'longitude']]

In [9]:
%%time
bandwidth = 0.1
ms = MeanShift(bandwidth = bandwidth)
ms.fit(exp_data)

CPU times: user 4min 4s, sys: 336 ms, total: 4min 5s
Wall time: 4min 5s


In [11]:
%%time
res = ms.predict(data[['latitude', 'longitude']])

CPU times: user 21.7 s, sys: 32.6 s, total: 54.3 s
Wall time: 14.5 s


In [114]:

office_location = { 
    'Los Angeles' : [33.751277, -118.188740] , 
    'Miami' : [25.867736, -80.324116], 
    'London' : [51.503016, -0.075479],
    'Amsterdam' : [52.378894, 4.885084],
    'Beijing' : [39.366487, 117.036146],
    'Sydney' : [-33.868457, 151.205134]}
def distance(point1, point2):
    return math.sqrt((point1[0]-point2[0])*(point1[0]-point2[0]) + (point1[1]-point2[1])*(point1[1]-point2[1]))

def compute_table_for_each_office(clusters, office_location):
    index = range(len(clusters));
    df_ = pd.DataFrame(index=index, columns=office_location.keys())
    df_ = df_.fillna(0)
    for key in office_location:
        for i in index:
            df_.set_value(i, key, distance(clusters[i],office_location[key]))
    return df_;
def compute_min_for_each_office(clusters,table ,office_location):
    res = {}
    for key in office_location:
        indx = table[key].idxmin()
        table = table.drop(indx)
        res[key] = clusters[indx]
    return [res,table]

def show_res(df):
    for key in df:
        print df[key][0],df[key][1]
        print ''
    

In [117]:
%%time
table = compute_table_for_each_office(ms.cluster_centers_,office_location)
for i in range(4):
    df = compute_min_for_each_office(ms.cluster_centers_,table,office_location)[0]
    table = compute_min_for_each_office(ms.cluster_centers_,table,office_location)[1]
    show_res(df)



39.904214 116.407413

34.0603975546 -118.248709027

25.78581242 -80.2179380368

51.5029912609 -0.12553728871

-33.8606304286 151.204775929

52.3729639903 4.89231722258

39.121079 117.214389

34.0354869531 -118.438997719

26.1208626586 -80.1589066802

51.4267632889 -0.303732066667

-33.7149546 150.3114074

52.080058 5.10513275

40.376322 116.842992

34.1274022191 -118.351883725

26.0100982493 -80.1999905857

51.455041 -0.9690884

-34.00190615 151.12806905

52.0435286167 4.32027353333

31.230393 121.473704

33.8173064339 -117.891249171

26.3525711663 -80.0852787174

51.5064787667 -0.365177266667

-33.9522629 151.0321372

52.388501 4.63376546667

CPU times: user 252 ms, sys: 0 ns, total: 252 ms
Wall time: 238 ms
